In [3]:
spark = (
    SparkSession.builder
    .appName("Order-Funnel-Analytics")
    .config(
        "spark.jars.packages",
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.3,"
        "org.postgresql:postgresql:42.7.3"
    )
    .config("spark.sql.shuffle.partitions", "6")
    .getOrCreate()
)
spark

In [2]:
spark

NameError: name 'spark' is not defined

In [4]:
KAFKA_BROKERS = "broker2:29094"
VALID_ORDERS_TOPIC = "orders_valid"

orders_raw = (
    spark.readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_BROKERS)
    .option("subscribe", VALID_ORDERS_TOPIC)
    .option("startingOffsets", "earliest")
    .load()
)


In [5]:
orders = (
    orders_raw
    .selectExpr("CAST(value AS STRING) AS json")
    .selectExpr(
        """
        from_json(
            json,
            'order_id STRING,
             event_type STRING,
             current_state STRING,
             processed_time STRING'
        ) AS data
        """
    )
    .select("data.*")
    .withColumn("event_time", to_timestamp("processed_time"))
)

In [6]:
order_funnel = (
    orders
    .groupBy(
        window(col("event_time"), "1 minutes"),
        col("current_state")
    )
    .agg(count("*").alias("orders_count"))
    .select(
        col("window.start").alias("window_start"),
        col("window.end").alias("window_end"),
        col("current_state"),
        col("orders_count")
    )
)

In [9]:
query = (
    order_funnel
    .writeStream
    .foreachBatch(
        lambda df, _: (
            df.write
            .format("jdbc")
            .option(
                "url",
                "jdbc:postgresql://postgres:5432/ECOMMERCE"
            )
            .option("dbtable", "order_funnel_metrics")
            .option("user", "admin")
            .option("password", "admin")
            .option("driver", "org.postgresql.Driver")
            .mode("append")
            .save()
        )
    )
    .outputMode("update")
    .option("checkpointLocation", "/tmp/checkpoints/order_funnel")
    .start()
)

query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/venv/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/venv/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/socket.py", line 707, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 